# Phase 2
## Data Description
**What are the observations (rows) and the attributes (columns)?**

The rows are movies that won best picture.

The columns:
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

**Why was this dataset created?**

I created this dataset to observe how imdb ratings changed throughout the years for best picture winners.

**Who funded the creation of the dataset?**

The Oscars and IMDB

**What processes might have influenced what data was observed and recorded and what was not?**

It makes sense that IMDB makes their rating information public; however, I found it really difficult to find a database of movie gross revenue and budget which would be a more true indicator of the success of a movie. Similarly, the Oscar database only had information about oscar nominations, winners, and the year that they were nominated/won. I could not find data for how much money was spent on the movies' oscar campaigns. This lack of monetary data is likely due to Hollywood's tendency to keep that sort of data private/ not easily accessible.

**What preprocessing was done, and how did the data come to be in the form that you are using?**

The Best Picture Winners had to be scraped from a search in the Academy's database.

The IMDB data was originally two data tables (rankings data and movie title data) that were merged into one based on a "tconst" value.

**If people are involved, were they aware of the data collection and if so, what purpose did they expect the data to be used for?
Where can your raw source data be found, if applicable? Provide a link to the raw data (hosted in a Cornell Google Drive or Cornell Box).**

The Academy probably didn't expect me to use their query and then scrape the resulting search. Considering, the popularity of movies they probably just assumed people to use it for their own knowledge and not something as intensive as a data scrape. As for IMDB, their documentation and readily available tsv format proves to show that they probably expected people to use their data in a data science application.

Oscars data: http://awardsdatabase.oscars.org/

IMDB data: https://www.imdb.com/interfaces/

## Oscar Best Picture Data

In [1]:
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install lxml

In [2]:
import numpy as np
import pandas as pd

In [3]:
from bs4 import BeautifulSoup

In [4]:
with open("raw data\Search Results - Academy Awards Search _ Academy of Motion Picture Arts & Sciences.html") as file_reader:
    soup = BeautifulSoup(file_reader, "lxml")

In [5]:
years = soup.findAll("div", {"class": "result-group-title"})
titles = soup.findAll("div", {"class": "awards-result-film-title"})
    
best_pictures = []
for i in range(len(years)):
    year = years[i].get_text()[1:5]
    title = titles[i].get_text()[1:-1]
    best_pictures.append({"year": year, "title": title})

print("first 5 elements of best_pictures array:")
best_pictures[:5]

first 5 elements of best_pictures array:


[{'year': '1927', 'title': 'Wings'},
 {'year': '1928', 'title': 'The Broadway Melody'},
 {'year': '1929', 'title': 'All Quiet on the Western Front'},
 {'year': '1930', 'title': 'Cimarron'},
 {'year': '1931', 'title': 'Grand Hotel'}]

In [6]:
best_picture_titles = []
for title in titles:
    best_picture_titles.append(title.get_text()[1:-1])

In [7]:
len(best_picture_titles)

92

## IMDB Ratings

**title.ratings.tsv.gz** – Contains the IMDb rating and votes information for titles
- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

In [9]:
ratings=pd.read_csv('raw data/title.ratings.tsv',delimiter='\t',encoding='utf-8', index_col="tconst")
ratings.head()

,averageRating,numVotes
tconst,,
tt0000001,5.6,1611
tt0000002,6.0,198
tt0000003,6.5,1292
tt0000004,6.1,121
tt0000005,6.1,2061


**title.basics.tsv.gz** - Contains the following information for titles:
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

In [10]:
basics=pd.read_csv('title.basics.tsv.tsv',delimiter='\t',encoding='utf-8', index_col="tconst")
basics.head()

C:\Users\Hahnbee\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tconst,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [11]:
data = ratings.merge(basics, how="inner", left_on='tconst', right_on='tconst')

In [12]:
is_movie = data['titleType']=='movie'
movie_data = data[is_movie]

In [ ]:
best_picture_data = pd.DataFrame(columns = movie_data.columns)
filtered_movies = []
for index, row in movie_data.iterrows(): # all movies
    for bp in best_pictures: # best picture movies
        if row.primaryTitle == bp['title'] and row.startYear == bp['year']:
            filtered_movies.append(row)

In [ ]:
best_picture_imdb_data_0 = best_picture_data.append(filtered_movies, True)

**IMDB ratings of Best Picture Winners**

In [ ]:
best_picture_imdb_data_0

In [ ]:
# Get rid of duplicates
duplicateRows = best_picture_imdb_data_0[best_picture_imdb_data_0.duplicated(['originalTitle'])]
bp_dr = pd.merge(best_picture_imdb_data_0, duplicateRows, how='outer', indicator=True)
bp_imdb_data_1 = bp_dr[bp_dr._merge == 'left_only']

In [ ]:
# get rid of unnecessary columns
bp_winners = bp_imdb_data_1.drop(columns=['titleType', 'originalTitle', 'isAdult', 'endYear', 'runtimeMinutes', 'genres', '_merge'])

In [ ]:
print(bp_winners.head())

In [ ]:
bp_winners.to_csv('best_picture_winners', index=False)

**IMDB Ratings for Nominees**

In [4]:
with open("raw data\Best Picture Nominees.html") as file_reader:
    noms_soup = BeautifulSoup(file_reader, "lxml")

In [5]:
noms_years = noms_soup.findAll("div", {"class": "result-group"})

nominees = []
for obj in noms_years:
    year = obj.find("a").get_text()[0:4]
    noms = obj.findAll("div", {"class": "awards-result-film-title"})
    for n in noms:
        nominees.append({"year": year, "title": n.get_text()[1:-1]})

In [6]:
noms_df = pd.DataFrame(nominees, columns=["year", "title"])

In [7]:
noms_df.to_csv('best_picture_nominees', index=False)

In [13]:
filtered_noms = []
for index, row in movie_data.iterrows(): # all movies
    for bp in nominees: # best picture movies
        if row.primaryTitle == bp['title'] and row.startYear == bp['year']:
            filtered_noms.append(row)

In [14]:
imdb_noms = pd.DataFrame(columns = movie_data.columns)
bp_noms_imdb = imdb_noms.append(filtered_noms, True)

In [17]:
duplicateRows_noms

,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
224,7.1,7,movie,The Artist,The Artist,0,2011,\N,100,Thriller
263,8.0,34,movie,Apollo 13,Apollo 13,0,1995,\N,49,Documentary
269,7.9,13,movie,Spotlight,Spotlight,0,2015,\N,99,Drama


In [16]:
# Get rid of duplicates

duplicateRows_noms = bp_noms_imdb[bp_noms_imdb.duplicated(['originalTitle'])]
dr_noms = pd.merge(bp_noms_imdb, duplicateRows_noms, how='outer', indicator=True)
noms_imdb_wo_dups = dr_noms[dr_noms._merge == 'left_only']

In [18]:
# get rid of unnecessary columns
bp_noms = noms_imdb_wo_dups.drop(columns=['titleType', 'originalTitle', 'isAdult', 'endYear', 'runtimeMinutes', 'genres', '_merge'])

In [19]:
bp_noms.head()

,averageRating,numVotes,primaryTitle,startYear
0,8.3,727889,A Clockwork Orange,1971
1,8.0,37321,Fiddler on the Roof,1971
2,7.7,104990,The French Connection,1971
3,8.0,40109,The Last Picture Show,1971
4,7.2,4182,Nicholas and Alexandra,1971


In [20]:
bp_noms.to_csv('best_picture_nominees', index=False)